In [1]:
%pylab
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import datetime

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# List of columns which contains a Names
list_names = ['ProductName', 'CSF', 'CompanyName', 'BrandName', 'PrimaryCategory', 'SubCategory', 'ChemicalName']

# List of columns which contains a Dates
list_dates = ['InitialDateReported', 'MostRecentDateReported', 'DiscontinuedDate', 'ChemicalCreatedAt', 
              'ChemicalUpdatedAt', 'ChemicalDateRemoved']

# Column of ChemicalCount
chemicalcount_column = 'ChemicalCount'

# Column of CasNumber
casnumber_column = 'CasNumber'

# Column of CasId
casid_column = 'CasId'

# Column of SubCategoryId
subcategoryid_column = 'SubCategoryId'

# Date value which means NaT 
nat_date = datetime.date(1900,1,1)

In [19]:
# Load data from data/ folder
data = pd.read_csv('data/cscpopendata.csv', sep = ',', parse_dates = list_dates)

# pd.isna() --> for NaN
# <value> is pd.NaT --> for NaT

# Fill NaT with 01/01/2500
for date_column in list_dates :
    data[date_column] = data[date_column].apply(lambda x: nat_date if x is pd.NaT else x)
    

# Fill NA with ''
data = data.fillna('')

# To delete
data_copy = data.copy(deep = True)

In [4]:
data = data_copy.copy(deep = True)

In [20]:
data.head()

,CDPHId,ProductName,CSFId,CSF,CompanyId,CompanyName,BrandName,PrimaryCategoryId,PrimaryCategory,SubCategoryId,...,CasNumber,ChemicalId,ChemicalName,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,ChemicalCount
0,2,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,,,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),53,...,13463-67-7,6,Titanium dioxide,2009-06-17,2013-08-28,2011-02-01,2009-07-09,2009-07-09,1900-01-01,1
1,3,Glover's Medicated Shampoo,,,338,J. Strickland & Co.,Glover's,18,Hair Care Products (non-coloring),25,...,65996-92-1,4,Distillates (coal tar),2009-07-01,2009-07-01,1900-01-01,2009-07-01,2009-07-01,1900-01-01,2
2,3,Glover's Medicated Shampoo,,,338,J. Strickland & Co.,Glover's,18,Hair Care Products (non-coloring),25,...,140-67-0,5,Estragole,2009-07-01,2009-07-01,1900-01-01,2009-07-02,2009-07-02,1900-01-01,2
3,4,PRECISION GLIMMER EYE LINER-ALL SHADES �,,,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),46,...,13463-67-7,7,Titanium dioxide,2009-07-09,2013-08-28,1900-01-01,2009-07-09,2009-07-09,1900-01-01,1
4,5,AVON BRILLIANT SHINE LIP GLOSS-ALL SHADES �,,,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),52,...,13463-67-7,8,Titanium dioxide,2009-07-09,2013-08-28,2011-02-01,2009-07-09,2009-07-09,1900-01-01,1


In [21]:
# Other columns
list_ids = [ col for col in list(data.columns) 
                    if col not in (list_names + list_dates + [casnumber_column] + [chemicalcount_column]) ]

In [23]:
# Add to 'data' a column for each 'date_column' in 'list_dates' with the year and month of each date 
list_dates_extended = list_dates.copy()

for date_column in list_dates :
    data[date_column + '_Year'] = data[date_column].apply(lambda x: x.year)
    data[date_column + '_Month'] = data[date_column].apply(lambda x: x.month)
    
    list_dates_extended.append(date_column + '_Year')
    list_dates_extended.append(date_column + '_Month')

In [24]:
data.head()

,CDPHId,ProductName,CSFId,CSF,CompanyId,CompanyName,BrandName,PrimaryCategoryId,PrimaryCategory,SubCategoryId,...,MostRecentDateReported_Year,MostRecentDateReported_Month,DiscontinuedDate_Year,DiscontinuedDate_Month,ChemicalCreatedAt_Year,ChemicalCreatedAt_Month,ChemicalUpdatedAt_Year,ChemicalUpdatedAt_Month,ChemicalDateRemoved_Year,ChemicalDateRemoved_Month
0,2,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES,,,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),53,...,2013,8,2011,2,2009,7,2009,7,1900,1
1,3,Glover's Medicated Shampoo,,,338,J. Strickland & Co.,Glover's,18,Hair Care Products (non-coloring),25,...,2009,7,1900,1,2009,7,2009,7,1900,1
2,3,Glover's Medicated Shampoo,,,338,J. Strickland & Co.,Glover's,18,Hair Care Products (non-coloring),25,...,2009,7,1900,1,2009,7,2009,7,1900,1
3,4,PRECISION GLIMMER EYE LINER-ALL SHADES �,,,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),46,...,2013,8,1900,1,2009,7,2009,7,1900,1
4,5,AVON BRILLIANT SHINE LIP GLOSS-ALL SHADES �,,,4,New Avon LLC,AVON,44,Makeup Products (non-permanent),52,...,2013,8,2011,2,2009,7,2009,7,1900,1


In [25]:
# Create a data_aggregate with the sum of 'chemicalcount_column' of the following columns:
# 'list_dates', 'casid_column' and 'subcategoryid_column'
data_aggregated = pd.DataFrame(data.groupby(list_dates_extended + [casid_column] + [subcategoryid_column], as_index = False)
                               .agg({chemicalcount_column : 'sum'}))
data_aggregated.head()

,InitialDateReported,MostRecentDateReported,DiscontinuedDate,ChemicalCreatedAt,ChemicalUpdatedAt,ChemicalDateRemoved,InitialDateReported_Year,InitialDateReported_Month,MostRecentDateReported_Year,MostRecentDateReported_Month,...,DiscontinuedDate_Month,ChemicalCreatedAt_Year,ChemicalCreatedAt_Month,ChemicalUpdatedAt_Year,ChemicalUpdatedAt_Month,ChemicalDateRemoved_Year,ChemicalDateRemoved_Month,CasId,SubCategoryId,ChemicalCount
0,2009-06-17,2013-08-28,2011-02-01,2009-07-09,2009-07-09,1900-01-01,2009,6,2013,8,...,2,2009,7,2009,7,1900,1,656,53,1
1,2009-07-01,2009-07-01,1900-01-01,2009-07-01,2009-07-01,1900-01-01,2009,7,2009,7,...,1,2009,7,2009,7,1900,1,889,25,2
2,2009-07-01,2009-07-01,1900-01-01,2009-07-02,2009-07-02,1900-01-01,2009,7,2009,7,...,1,2009,7,2009,7,1900,1,293,25,2
3,2009-07-09,2013-08-28,1900-01-01,2009-07-09,2009-07-09,1900-01-01,2009,7,2013,8,...,1,2009,7,2009,7,1900,1,656,46,1
4,2009-07-09,2013-08-28,2010-11-01,2009-07-09,2009-07-09,1900-01-01,2009,7,2013,8,...,11,2009,7,2009,7,1900,1,656,48,1


In [28]:
data_aggregated.shape

(17375, 21)

In [27]:
data.shape

(96896, 34)